<a href="https://colab.research.google.com/github/faiz-aljohani/ICS483-Project/blob/main/ICS483_PP2_DeepUPE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wget

!git clone https://github.com/hermosayhl/DeepUPE_pytorch.git
!mv /content/DeepUPE_pytorch/* /content
!rm -r /content/DeepUPE_pytorch

  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=a1a5add566001e7e42f7cdb2baab902bfa0e539ba1682bc70ee7e01ba6156aba
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget
Cloning into 'DeepUPE_pytorch'...
remote: Enumerating objects: 178, done.
remote: Total 178 (delta 0), reused 0 (delta 0), pack-reused 178
Receiving objects: 100% (178/178), 283.62 MiB | 36.93 MiB/s, done.
Resolving deltas: 100% (44/44), done.
Updating files: 100% (94/94), done.


In [ ]:
!pip install imageio
!pip install Cython
!pip install numpy
!pip install dill
!pip install Pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 2.7 MB/s eta 0:00:00


In [ ]:
!gdown 1dzuLCk9_gE2bFF222n3-7GVUlSVHpMYC
!unzip LOL-v2.zip

In [ ]:
!rm /content/sample_results/*
!rm /content/high_resolution/input/*

In [ ]:
!mv /content/LOL-v2/Real_captured/Test/Low/* /content/high_resolution/input/

In [ ]:
!python /content/inference.py

luma_bins  :  8
channel_multiplier  :  1
spatial_bin  :  8
batch_norm  :  True
low_size  :  256
full_size  :  512
use_cuda  :  True
eps_value  :  0.0001
save_extra  :  False
images_dir  :  ./high_resolution/input
result_dir  :  ./sample_results
checkpoints_file  :  ./checkpoints/simple_batch_16/epoch_48_psnr_23.677.pth
loaded weights from ./checkpoints/simple_batch_16/epoch_48_psnr_23.677.pth
100 images are to be processed !
1/100===> processing ./high_resolution/input/low00693.png and saved to ./sample_results/low00693.png
2/100===> processing ./high_resolution/input/low00782.png and saved to ./sample_results/low00782.png
3/100===> processing ./high_resolution/input/low00787.png and saved to ./sample_results/low00787.png
4/100===> processing ./high_resolution/input/low00777.png and saved to ./sample_results/low00777.png
5/100===> processing ./high_resolution/input/low00744.png and saved to ./sample_results/low00744.png
6/100===> processing ./high_resolution/input/low00780.png and save

In [ ]:
!pip install torcheval

import torch
import torchvision.transforms as T
import os
from torcheval.metrics import PeakSignalNoiseRatio, MeanSquaredError
from torchvision.transforms.functional import to_tensor
from skimage.metrics import structural_similarity as ssim
from PIL import Image
import re
import cv2
import numpy as np

def prepare_data(enhanced_dir_path, groundtruth_dir_path):
    # Custom folder preparation and filtering
    enhanced_images = sorted(os.listdir(enhanced_dir_path))
    enhanced_images = [enhanced_dir_path + "/" +imgName for imgName in enhanced_images]
    groundtruth_images = sorted(os.listdir(groundtruth_dir_path))
    groundtruth_images = [groundtruth_dir_path + "/" + imgName for imgName in groundtruth_images]
    return enhanced_images, groundtruth_images

def evaluate_psnr(enhanced_images, groundtruth_images):
    transform = T.ToTensor()
    metric = PeakSignalNoiseRatio()
    eval_list = []

    for i in range(len(enhanced_images)):
        enhanced_img = Image.open(enhanced_images[i])
        groundtruth_img = Image.open(groundtruth_images[i])

        tensor_enhanced_img = transform(enhanced_img)
        tensor_groundtruth_img = transform(groundtruth_img)

        metric.update(tensor_enhanced_img, tensor_groundtruth_img)
        eval_list.append(metric.compute().item())

    result = sum(eval_list) / len(eval_list)
    return result


def evaluate_ssim(enhanced_images, groundtruth_images):
  eval_list = []

  for i in range(len(enhanced_images)):
    enhanced_img = Image.open(enhanced_images[i])
    groundtruth_img = Image.open(groundtruth_images[i])

    enhanced_np = np.array(enhanced_img)
    groundtruth_np = np.array(groundtruth_img)

    ssim_for_each_channel = []
    for channel in range(3):
      ssim_score_channel = ssim(enhanced_np[:, :, channel], groundtruth_np[:, :, channel])
      ssim_for_each_channel.append(ssim_score_channel)

    avg_ssim_channels = np.mean(ssim_for_each_channel)
    eval_list.append(avg_ssim_channels)

  result = np.mean(eval_list)
  return result


def evaluate_mse(enhanced_images, groundtruth_images):
  eval_list = []

  for i in range(len(enhanced_images)):
    enhanced_img = np.array(Image.open(enhanced_images[i]))
    groundtruth_img = np.array(Image.open(groundtruth_images[i]))

    mse = np.mean(np.square(enhanced_img - groundtruth_img))
    eval_list.append(mse)

  result = np.mean(eval_list)
  return result


def evaluate_methods(enhanced_dir_path, groundtruth_dir_path, method):
    enhanced_images, groundtruth_images = prepare_data(enhanced_dir_path, groundtruth_dir_path)
    if method == "PSNR":
        result = evaluate_psnr(enhanced_images, groundtruth_images)
    elif method == "SSIM":
        result = evaluate_ssim(enhanced_images, groundtruth_images)
    elif method == "MSE":
        result = evaluate_mse(enhanced_images, groundtruth_images)
    else:
        raise ValueError("Unsupported evaluation method.")

    return result

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 4.7 MB/s eta 0:00:00


In [ ]:
enhanced_dir_path = "/content/sample_results"
groundtruth_dir_path = "/content/LOL-v2/Real_captured/Test/Normal"

methods = ["PSNR","SSIM","MSE"]
for method in methods:
    result = evaluate_methods(enhanced_dir_path, groundtruth_dir_path, method)
    print("The {} for the test images is: {:.2f}".format(method, result))

The PSNR for the test images is: 12.47
The SSIM for the test images is: 0.47
The MSE for the test images is: 111.33
